In [9]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from scipy.io import loadmat
from scipy.stats import zscore
import ast
from utils.plot_utils import combine_pdf_big
from utils.beh_functions import session_dirs, parseSessionID, load_model_dv, makeSessionDF, get_session_tbl, get_unit_tbl, get_history_from_nwb
from utils.ephys_functions import*
from utils.lick_utils import load_licks
from utils.combine_tools import apply_qc, to_str_intlike

from open_ephys.analysis import Session
from pathlib import Path
import glob

import json
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import re
from aind_dynamic_foraging_basic_analysis.plot.plot_foraging_session import plot_foraging_session
from aind_dynamic_foraging_data_utils.nwb_utils import load_nwb_from_filename
from hdmf_zarr.nwb import NWBZarrIO

import pandas as pd
import pickle
import scipy.stats as stats
from joblib import Parallel, delayed
from multiprocessing import Pool
from functools import partial
import time
import shutil 
from aind_ephys_utils import align
%matplotlib inline

In [10]:
target_folder = '/root/capsule/scratch/manuscript/F_spontlicks'

In [11]:
dfs = [pd.read_csv('/root/capsule/code/data_management/session_assets.csv'),
        pd.read_csv('/root/capsule/code/data_management/hopkins_session_assets.csv')]
df = pd.concat(dfs)
session_list = df['session_id'].values.tolist()
ani_list = [str(session).split('_')[1] for session in session_list if str(session).startswith('behavior')]
session_list = [session for session in session_list if str(session).startswith('behavior')]

In [15]:
video_data_file = '/root/capsule/data/all_tongue_movements_04022026/all_tongue_movements_04022026.parquet'
all_lick_df = pd.read_parquet(video_data_file)
session_video_list = all_lick_df['session'].unique().tolist()
del all_lick_df
print(session_video_list)

['behavior_716325_2024-05-31_10-31-14', 'behavior_751004_2024-12-20_13-26-07', 'behavior_751004_2024-12-21_13-28-24', 'behavior_751004_2024-12-22_13-09-11', 'behavior_751004_2024-12-23_14-19-57', 'behavior_751181_2025-02-25_12-12-30', 'behavior_751181_2025-02-27_11-24-44', 'behavior_751766_2025-02-11_11-53-32', 'behavior_751766_2025-02-13_11-31-21', 'behavior_751766_2025-02-14_11-37-11', 'behavior_751766_2025-02-15_12-08-11', 'behavior_751769_2025-01-16_11-31-52', 'behavior_751769_2025-01-17_11-37-35', 'behavior_751769_2025-01-18_10-15-21', 'behavior_752014_2025-03-26_11-18-52', 'behavior_752014_2025-03-27_12-03-19', 'behavior_752014_2025-03-28_11-04-56', 'behavior_754897_2025-03-11_12-07-35', 'behavior_754897_2025-03-12_12-23-09', 'behavior_754897_2025-03-14_11-28-48', 'behavior_754897_2025-03-15_11-32-14', 'behavior_758017_2025-02-05_11-42-30', 'behavior_758017_2025-02-06_11-26-09', 'behavior_758017_2025-02-07_14-11-05', 'behavior_758018_2025-03-19_11-16-41', 'behavior_758018_2025-03

In [22]:
pdf_combine_folder = os.path.join(target_folder, 'pdf_combine')
if os.path.exists(pdf_combine_folder):
    shutil.rmtree(pdf_combine_folder)
if not os.path.exists(pdf_combine_folder):
    os.makedirs(name=pdf_combine_folder, exist_ok=True)
for session in session_list:
    # if find a session_video_list, move video pdf to target folder
    # print(f"Processing session {session}...")
    animal_id, session_time_curr, _  = parseSessionID(session)
    session_list_curr = [s for s in session_video_list if str(s).startswith(f'behavior_{animal_id}')]
    if len(session_list_curr) == 0:
        continue
    else:
        session_index = np.argmin([abs((parseSessionID(s)[1] - session_time_curr).total_seconds()) for s in session_list_curr])
        if np.min([abs((parseSessionID(s)[1] - session_time_curr).total_seconds()) for s in session_list_curr])>60:
            print(f"No closely matched session found for {session}. Skipping.")
        else:

            print(f"Found matching session {session_list_curr[session_index]} for {session}. Moving PDF.")
            session_dir = session_dirs(session)
            pdf_file = os.path.join(session_dir['beh_fig_dir'], f'{session}_lick_video_analysis_combined.pdf')
            if not os.path.exists(pdf_file):
                print(f"PDF file {pdf_file} does not exist. Skipping.")
                continue
            target_file = os.path.join(pdf_combine_folder, f'{session}_lick_video_analysis_combined.pdf')
            if os.path.exists(target_file):
                print(f"Target file {target_file} already exists. Skipping.")
                continue
            shutil.copy(pdf_file, target_file)


No closely matched session found for behavior_716325_2024-05-29_10-33-32. Skipping.
No closely matched session found for behavior_716325_2024-05-30_11-33-46. Skipping.
Found matching session behavior_716325_2024-05-31_10-31-14 for behavior_716325_2024-05-31_10-31-14. Moving PDF.
No closely matched session found for behavior_716325_2024-06-01_09-25-37. Skipping.
No closely matched session found for behavior_751004_2024-12-19_11-50-37. Skipping.
Found matching session behavior_751004_2024-12-20_13-26-07 for behavior_751004_2024-12-20_13-26-11. Moving PDF.
Found matching session behavior_751004_2024-12-21_13-28-24 for behavior_751004_2024-12-21_13-28-28. Moving PDF.
Found matching session behavior_751004_2024-12-22_13-09-11 for behavior_751004_2024-12-22_13-09-17. Moving PDF.
Found matching session behavior_751004_2024-12-23_14-19-57 for behavior_751004_2024-12-23_14-20-03. Moving PDF.
Found matching session behavior_751769_2025-01-16_11-31-52 for behavior_751769_2025-01-16_11-32-05. Movi